In [10]:
import pandas as pd
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
import numpy as np

In [11]:
label = pd.read_csv('../../data/comp5046-labels.csv',header=None) # Read the label
Lr_label = list(set(label[2].values)) # Unique the label

def prediction(x): # Function of prediction
    tp = []
    for w in W:
        tp.append(x.dot(w))
    predict_num = tp.index(max(tp)) # Found the max value of x dot w
    return predict_num


folds = list(range(10)) # How many fold here
text_len = 2700 # How many training article 
num_of_epoch = 5 # The number of epoch

for fold in folds:
    step = (num_of_epoch*(2700))
    total_step = step
    W = [np.zeros(104317) for i in range(len(Lr_label))] # W value
    a = [np.zeros(104317) for i in range(len(Lr_label))] # Averaged W 
    for epoch in range(num_of_epoch):
        number_of_training = 0
        predict_number_list = list(range(200 * fold, 200 * (fold + 1)))
        for text in pd.read_csv('result/single_data.csv',header=None,chunksize=600): # Read each 600 article
            label = pd.read_csv('../../data/comp5046-labels.csv',header=None)
            text = pd.merge(text,label[[1,2]],left_on=104317,right_on=1) # Merge the value and aricle

            y = [Lr_label.index(it) for it in text['2_y'].values]
   
            X = text.values[:,:-3]
            for each_data in range(len(X)): # Training the data
                if number_of_training not in predict_number_list:
                    number_of_training+=1
                    y_ = prediction(X[each_data])
                    true_y = y[each_data]
                    step -=1
                    if y_ != true_y:
                        W[true_y] = W[true_y] + X[each_data]
                        W[y_] = W[y_] - X[each_data]
                        a[true_y] = a[true_y] + (X[each_data])*(step/(num_of_epoch*text_len))
                        a[y_] = a[y_] - (X[each_data])*(step/(num_of_epoch*text_len))
                        print('*'*(100-int(100*step/total_step))+'-'*int((step/total_step)*100)+' '+str(int(100-(step/total_step)*100)),end='\r',flush=True)                
                else:
                    number_of_training+=1
    # Prediction
        
    W = a
    
    number_of_training = 0
    total_y_ = []
    total_y = []

    total_y_label = [Lr_label[i] for i in total_y]
    total_y_label_ = [Lr_label[i] for i in total_y_]
    predict_number_list = list(range(200 * fold, 200 * (fold + 1)))
    for text in pd.read_csv('result/single_data.csv',header=None,chunksize=600):
        label = pd.read_csv('../../data/comp5046-labels.csv',header=None)
        text = pd.merge(text,label[[1,2]],left_on=104317,right_on=1)

        y = [Lr_label.index(it) for it in text['2_y'].values]

        X = text.values[:,:-3]
        for each_data in range(len(X)):

            if number_of_training in predict_number_list and number_of_training < 200* (fold + 1):
                number_of_training+=1
                y_ = prediction(X[each_data])
                true_y = y[each_data]

                total_y_.append(y_)
                total_y.append(true_y)
            elif number_of_training not in predict_number_list and number_of_training < 200 * fold:
                number_of_training+=1
            elif number_of_training not in predict_number_list and number_of_training > 200 * (fold + 1):
                number_of_training+=1
                print('Testing is over.')
                break

        if len(total_y_) == 200: # Save the perfomance of result
            total_y_label = [Lr_label[i] for i in total_y]
            total_y_label_ = [Lr_label[i] for i in total_y_]
            print("Prediction score:\n%s" % metrics.classification_report(total_y_label, total_y_label_))
            result = metrics.classification_report(total_y_label, total_y_label_)
            macro = ' '.join(list(map(str,metrics.precision_recall_fscore_support(total_y_label,total_y_label_,average='macro'))))
            micro = ' '.join(list(map(str,metrics.precision_recall_fscore_support(total_y_label,total_y_label_,average='micro'))))
            with open('result_%s.csv' % fold, 'w') as file:  # Save the metrics result to file
                for line in result:
                    file.write(line)
                file.write('\n macro: '+macro)
                file.write('\n micro: '+micro)
            print('Save the result.')
            break



epoch:  0 Finished trainig.------------------------------------------------------------------------- 19
epoch:  1 Finished trainig.*************------------------------------------------------------------ 39
epoch:  2 Finished trainig.********************************----------------------------------------- 58
epoch:  3 Finished trainig.****************************************************--------------------- 78
epoch:  4 Finished trainig.***********************************************************************-- 97


/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Prediction score:
                       precision    recall  f1-score   support

            Education       0.95      0.95      0.95        21
        Entertainment       0.84      0.89      0.86        46
                Error       0.25      0.22      0.24         9
              Finance       0.67      1.00      0.80         2
          Hospitality       0.00      0.00      0.00         1
                Other       0.00      0.00      0.00         2
Public administration       0.67      0.64      0.65        28
          Real estate       0.00      0.00      0.00         1
              SciTech       0.67      0.29      0.40         7
               Social       0.60      0.75      0.67        20
               Sports       0.88      0.93      0.91        46
                Trade       0.55      0.55      0.55        11
            Transport       1.00      0.67      0.80         6

          avg / total       0.75      0.77      0.75       200

Save the result.
epoch:  0 Finishe

/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Prediction score:
                       precision    recall  f1-score   support

            Education       0.87      0.96      0.92        28
        Entertainment       0.81      0.96      0.88        50
                Error       0.29      0.10      0.14        21
              Finance       1.00      0.50      0.67         4
               Health       0.00      0.00      0.00         1
          Hospitality       0.00      0.00      0.00         0
                Other       0.00      0.00      0.00         1
Public administration       0.65      0.76      0.70        17
          Real estate       0.00      0.00      0.00         1
              SciTech       1.00      0.20      0.33         5
               Social       0.33      0.33      0.33        18
               Sports       0.84      0.86      0.85        37
                Trade       0.67      0.91      0.77        11
            Transport       0.75      1.00      0.86         6

          avg / total       0.70   